In [ ]:
import cv2


In [ ]:
# Load the image in color format
img = cv2.imread('./assets/nature.jpg', cv2.IMREAD_COLOR)

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Resize the image to a manageable size
resized = cv2.resize(gray, (512, 512))

# Display the original and preprocessed images
cv2.imshow('Original Image', img)
cv2.imshow('Preprocessed Image', resized)
cv2.waitKey(0)
cv2.destroyAllWindows()